In [2]:
%matplotlib inline

from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Alphabet import IUPAC

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestClassifier

import shap

In [3]:
#pd.set_option('display.max_rows', 500)

In [8]:
fasta_file = '../DATA/ZIBRA 2/FUNED_AGOSTO-2018/CONSENSUS/CONSENSUS INDIVIDUAIS/YFV_FUNED_AGO_18_consensus.fas'

### different CTs

In [9]:
# Gets the sequences IDs
identifiers = [seq_record.id for seq_record in SeqIO.parse(fasta_file, "fasta")]

In [10]:
# Gets the sequences nucleotides
seqs = np.array([list(str(seq_rec.seq)) for seq_rec in SeqIO.parse(fasta_file, "fasta")])

In [11]:
# Creates columns names based on position, starting from 1, to make it consistent with the 
# sequence analysis, which starts at base number 1.
cols = list(range(1, seqs.shape[1]+1))

In [12]:
# Creates dataframe with data
df = pd.DataFrame(seqs, index=identifiers, columns=cols)
df.head()

,1,2,3,4,5,6,7,8,9,10,...,10230,10231,10232,10233,10234,10235,10236,10237,10238,10239
FAH50662/BC33_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
FAH50662/BC34_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
FAH50662/BC18_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
FAH50662/BC19_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
FAH50662/BC20_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N


In [18]:
samples = pd.read_excel('../DATA/ZIBRA 2/FUNED_AGOSTO-2018/Felipe_Iani_primatas.xlsx', index_col='Barcode Number')
samples = samples[samples["Virus"]=="YFV"]
samples = samples[['Original_Lab_CT value', 'Host', 'Collection_Date']]
samples.head()

,Original_Lab_CT value,Host,Collection_Date
Barcode Number,,,
BC29,9.95,Cebus apella,2017-03-24 00:00:00
BC18,5.54,Alouatta guariba clamitans,2017-03-24 00:00:00
BC19,6.62,cebidae,2017-04-03 00:00:00
BC26,8.74,Callithrix,2017-04-03 00:00:00
BC27,8.78,Alouatta clamitans,2017-03-13 00:00:00


In [19]:
import re
#pattern = "^\d+"
#dic={}

In [20]:
#df['ID'] = ''
#df['species'] = ''
#df['date'] = ''
#df['Ct'] = ''

In [21]:
for index_a, sample_a in samples.iterrows():
    #print(index_a)
    pattern = index_a
    regex = re.compile(pattern)
    for index_b, sample_b in df.iterrows():
        if regex.search(index_b):
            print(pattern, index_a, index_b)
            df.loc[index_b,'Barcode Number'] = index_a
            df.loc[index_b,'Host'] = samples.loc[index_a, 'Host']
            df.loc[index_b,'Date'] = samples.loc[index_a, 'Collection_Date']
            df.loc[index_b,'Ct'] = samples.loc[index_a, 'Original_Lab_CT value']

BC29 BC29 FAH50662/BC29_FAH50662.primertrimmed.sorted.bam
BC18 BC18 FAH50662/BC18_FAH50662.primertrimmed.sorted.bam
BC19 BC19 FAH50662/BC19_FAH50662.primertrimmed.sorted.bam
BC26 BC26 FAH50662/BC26_FAH50662.primertrimmed.sorted.bam
BC27 BC27 FAH50662/BC27_FAH50662.primertrimmed.sorted.bam
BC31 BC31 FAH50662/BC31_FAH50662.primertrimmed.sorted.bam
BC24 BC24 FAH50662/BC24_FAH50662.primertrimmed.sorted.bam
BC22 BC22 FAH50662/BC22_FAH50662.primertrimmed.sorted.bam
BC28 BC28 FAH50662/BC28_FAH50662.primertrimmed.sorted.bam
BC23 BC23 FAH50662/BC23_FAH50662.primertrimmed.sorted.bam
BC34 BC34 FAH50662/BC34_FAH50662.primertrimmed.sorted.bam
BC21 BC21 FAH50662/BC21_FAH50662.primertrimmed.sorted.bam
BC30 BC30 FAH50662/BC30_FAH50662.primertrimmed.sorted.bam
BC32 BC32 FAH50662/BC32_FAH50662.primertrimmed.sorted.bam
BC33 BC33 FAH50662/BC33_FAH50662.primertrimmed.sorted.bam
BC20 BC20 FAH50662/BC20_FAH50662.primertrimmed.sorted.bam


In [22]:
df

,1,2,3,4,5,6,7,8,9,10,...,10234,10235,10236,10237,10238,10239,Barcode Number,Host,Date,Ct
FAH50662/BC33_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC33,alouatta caraya,2017-12-04,12.35
FAH50662/BC34_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC34,Callithrix,2017-04-22,12.68
FAH50662/BC18_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC18,Alouatta guariba clamitans,2017-03-24,5.54
FAH50662/BC19_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC19,cebidae,2017-04-03,6.62
FAH50662/BC20_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC20,?????,2018-01-15,6.98
FAH50662/BC21_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC21,alouatta caraya,2017-05-05,7.46
FAH50662/BC22_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC22,alouatta caraya,2017-04-06,7.87
FAH50662/BC23_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC23,cebidae,2017-04-12,8.00
FAH50662/BC24_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC24,alouatta caraya,2017-04-03,8.64
FAH50662/BC26_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC26,Callithrix,2017-04-03,8.74


In [23]:
df_Callithrix = df[df['Host'] == 'Callithrix']
df_Callithrix

,1,2,3,4,5,6,7,8,9,10,...,10234,10235,10236,10237,10238,10239,Barcode Number,Host,Date,Ct
FAH50662/BC34_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC34,Callithrix,2017-04-22,12.68
FAH50662/BC26_FAH50662.primertrimmed.sorted.bam,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,BC26,Callithrix,2017-04-03,8.74
